In [1]:
using Pkg, RData

cd("C:\\Users\\UOS\\Desktop\\portfolio\\dataset")

df_kospi = load("kospi_1018.RData")

typeof(df_kospi)

keys(df_kospi) ### Dict type의 data에 key를 확인

kospi_mat = df_kospi["df_kospi_1018"][2:end,]
group_inx = df_kospi["group_index"]

### Data type 변환 DF -> Matrix
kospi_mat = convert(Matrix, kospi_mat);

### log return으로  변환
log_return_mat= function(x)
    
    nrow = size(x)[1]
    ncol = size(x)[2]
    
    lag1 = x[2:end,:]
    
    log_return_mat = log.(lag1) .- log.(kospi_mat[1:(end-1),:])
    
    return log_return_mat
end

┌ Warning: 13 element in a 2 list, assuming it's the last element
└ @ RData C:\Users\UOS\.julia\packages\RData\QWOSZ\src\readers.jl:138
┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[1]:10
└ @ Core In[1]:10


#3 (generic function with 1 method)

In [2]:
rmat = 100 * log_return_mat(kospi_mat)[1:240,:]

using Statistics

μ̂ = mean(rmat, dims = 1)

nrow = size(rmat)[1]
ncol = size(rmat)[2]

p = ncol; n = nrow

In [3]:
mean(μ̂)

0.08383780003373852

In [ ]:
using Random, Distributions, LinearAlgebra

In [4]:
μ₀ = 0.05

Amat = vcat(Diagonal(ones(p)), Diagonal(ones(p)), ones(p)', μ̂)
Bmat = vcat(Diagonal(-1*ones(2*p)), zeros(2*p)', zeros(2*p)')
Cmat = vcat(zeros(2*p), 1, μ₀)

X̃ = hcat(fill(-1, n), rmat)
K = hcat(zeros(p), Diagonal(ones(p)));

In [5]:
using StatsBase

In [16]:
ρ = 10; τ = 0.1

tmp_z = fill(0, 2*p)/p
tmp_u = fill(0, 2*p+2)
tmp_β = fill(1, p)/p
tmp_β₀ = 1e-5

tmp_β̃ = vcat(tmp_β₀, tmp_β);

pl = sqrt.(counts(group_inx)); iter_outer = 1000; iter_inner = 100; 

λ₁ = 0.001; λ₂ = 0.001;

let i = 1

    while i <= iter_outer

        let j = 1 

            while j <= iter_inner  

                tmp_W = Diagonal(ifelse.(abs.((X̃ * tmp_β̃) * 4) .< 1e-8, 1e+8, abs.(1 ./ ((X̃ * tmp_β̃)*4))))

                tmp_β̃ = (-1/2) * inv(X̃' * tmp_W * X̃ + (ρ/2) * (Amat * K)' * (Amat * K)) * ((τ - (1/2)) * X̃' * ones(n) +
                    ρ * (Amat*K)' * ((1/ρ)*tmp_u + Bmat*tmp_z - Cmat)) 
                j += 1
            end
        end
        
        tmp_β = tmp_β̃[2:end]

        v = (Amat * tmp_β - Cmat)
        v₁ = v[1:p]
        tmp_z₁ = v₁ + tmp_u[1:p]/ρ

        ### Proximal operator
        tmp_z₁ = ifelse.(abs.(tmp_z₁) .<= λ₁, 0, tmp_z₁ - λ₁ * sign.(tmp_z₁))

        v₂ = v[(p+1):(2*p)]
        u₂ = tmp_u[(p+1):(2*p)]

        tmp_z₂ = v₂ + u₂/ρ 

        for l in sort(unique(group_inx))
            plₗ = pl[l]
            v₂ₗ = v₂[group_inx .== l]
            u₂ₗ = u₂[group_inx .== l]

            if(norm(tmp_z₂[group_inx .== l], 2) .> λ₂ * plₗ)
                tmp_z₂[group_inx .== l] = tmp_z₂[group_inx .== l] - λ₂ * plₗ * tmp_z₂[group_inx .== l] / norm(tmp_z₂[group_inx .== l],2)
            else 
                tmp_z₂[group_inx .== l] .= 0
            end
        end        

        tmp_z = vcat(tmp_z₁, tmp_z₂)

        tmp_u = tmp_u + ρ * (Amat * tmp_β + Bmat * tmp_z - Cmat)

        i += 1

    end
    
    return tmp_β̃
end

163-element Array{Float64,1}:
 -0.17860593323067295  
 -0.017699005277845486 
  0.01686876432788773  
 -0.0007918409608063067
  0.027119200970871775 
 -0.03411985180147096  
  0.06280498789578902  
 -0.020516205246898456 
 -0.06217333351266974  
 -0.0294871887528316   
  0.050855299908182015 
 -0.02022207995404149  
  0.09389297968427006  
  ⋮                    
 -0.1071550247022715   
 -0.05238475012883247  
  0.007759504334636041 
  0.0034761104007631598
  0.014562064221140569 
  0.018930465161727642 
  0.0014825569178330148
 -0.029776386461299428 
  4.400779126282478e-6 
 -0.0007993062884492301
  0.004548163282594424 
  0.009004989458189575 

In [17]:
sum(tmp_β̃[2:end])

1.0000000236258373

In [18]:
μ̂ * tmp_β̃[2:end]

1-element Array{Float64,1}:
 0.04999990166935911